# DEG Analays

In [4]:
import scanpy as sc
import pandas as pd

# from pydeseq2.dds import DeseqDataSet
# from pydeseq2.ds import DeseqStats
# import decoupler as dc

import os
from pathlib import Path

BASE_PATH = os.getenv("BASE_PATH_FOR_SAVING")
DISEASED_SAMPLES_FOLDER = f"{BASE_PATH}/diseased"
MSN_BICAN_FOLDER = "/home/gdallagl/myworkdir/XDP/data/STR_zonation_references_h5ad"

# Cell type annotation to use for DEG
CT_ANNOTATION_FOR_DEG = ""

# Create Cell Type specific adata

Create adata obj with both dieseased anf helathy sample but for a single cell type.

Helthy and diseased samples are saved in differt locations and have differ nomenclture for columns.

In addition, diseased are split btu sample, while healthy are already split by cell type but not by sample.

### Diseased

In [ ]:
adata_diseased_list = []
sample_diseased_list = []

# Load Diseased data (each adata is a differt sample/patient)
for sample_folder in [p for p in Path(DISEASED_SAMPLES_FOLDER).iterdir() if p.is_dir()]:

    adata_path = sample_folder / "adata" / "all_lib_adata_zoned.h5ad"

    adata_tmp = sc.read_h5ad(adata_path, backed="r")

    # print(adata_tmp.obs.columns)
    # display(adata_tmp.obs)
    # print(sample_folder)

    # restart from raw counts
    adata_minimal = sc.AnnData(
        X=adata_tmp.layers["counts"].copy(),                   
        obs=adata_tmp.obs.copy(),
        var=adata_tmp.var.copy()
    )
    del adata_tmp

    # add sample:
    sample_diseased_list.append(sample_folder.name)

    # add ot list
    adata_diseased_list.append(adata_minimal)

    #break

adata_diseased_list

In [ ]:
# Save var info from first dataset before concat
# So all dataset will have same metadata for genes
var_info = adata_diseased_list[0].var[['feature_types', 'genome', 'gene_symbol']].copy()

# Concate
all_adata_diseased = sc.concat(adata_diseased_list, 
                  axis=0, # concatenate along observations, i.e., stacking cells)
                  index_unique='-' , # causes new barcodes to be orig_idx + '-' + key
                  keys=sample_diseased_list,
                  label="donor_id",
                  join='inner')  # keeps COMMON genes
del adata_diseased_list

# Restore var info for common genes
all_adata_diseased.var = var_info.loc[all_adata_diseased.var_names]

print(all_adata_diseased)
display(all_adata_diseased.obs.head(3))
display(all_adata_diseased.var.head(3))
assert all_adata_diseased.obs_names.is_unique, "ERROR: all_adata_diseased.obs_names are not unique!"


NameError: name 'adata_diseased_list' is not defined

### Healthy

In [ ]:
adata_healthy_list = []

# Load Helahy data (each adata is a differt cell type)
for adata_path in [p for p in Path(MSN_BICAN_FOLDER).iterdir() if not p.is_dir()]:

    adata_tmp = sc.read_h5ad(adata_path)#, backed="r")

    # print(adata_tmp.obs.columns)
    # display(adata_tmp.obs)
    # print(sample_folder)

    # restart from raw counts
    adata_minimal = sc.AnnData(
        X=adata_tmp.X.copy(),           # raw counts in X       
        obs=adata_tmp.obs.copy(),
        var=adata_tmp.var.copy()
    )
    del adata_tmp

    # add ot list
    adata_healthy_list.append(adata_minimal)

adata_healthy_list

In [ ]:

# Concate
all_adata_healthy = sc.concat(adata_healthy_list, 
                  axis=0, # concatenate along observations, i.e., stacking cells)
                  index_unique='-' , # causes new barcodes to be orig_idx + '-' + key
                #   keys=sample_diseased_list,
                #   label="donor_id",
                  join='inner')  # keeps COMMON genes
del adata_healthy_list

print(all_adata_healthy)
display(all_adata_healthy.obs.head(3))
display(all_adata_healthy.var.head(3))
assert all_adata_healthy.obs_names.is_unique, "ERROR: all_adata_healthy.obs_names are not unique!"


In [30]:
t = sc.read_h5ad("/home/gdallagl/myworkdir/XDP/data/STR_zonation_references_h5ad/STRd_D1_Striosome_MSN.h5ad", backed="r")
t.X[:100, :100].toarray().sum(axis=1)

array([301., 171., 130., 175., 325., 164., 383., 241., 143., 336., 260.,
       231., 227., 131., 290., 277., 306., 253.,  61., 173., 313., 398.,
       219., 307., 383., 285., 224., 211., 234., 198., 214., 241., 226.,
       318., 300., 162., 321., 282., 179., 303., 424., 244., 329., 321.,
       286., 123., 191., 213., 236., 317., 185., 200., 216., 245., 139.,
        77., 203., 335., 144., 283., 209., 307., 225., 380., 379., 208.,
       255., 199., 336.,  82., 201., 257., 162., 229., 299., 189., 211.,
       362., 311.,  90., 208., 189., 352., 293., 381., 314., 153., 237.,
       285., 408., 340., 287., 186., 292., 268., 242., 234., 199., 185.,
       259.])

### Merge All

In [ ]:
important_columns = ["donor_id", "library", "log1p_n_genes_by_counts", "log1p_total_counts_mt", "zone"]


# Save var info from first dataset before concat
# So all dataset will have same metadata for genes
var_info = all_adata_diseased.var[['feature_types', 'genome', 'gene_symbol']].copy()

# Concate
adata = sc.concat([all_adata_diseased, all_adata_healthy], 
                  axis=0, # concatenate along observations, i.e., stacking cells)
                  index_unique='-' , # causes new barcodes to be orig_idx + '-' + key
                  keys=["diseased", "healthy"],
                  label="state",
                  join='inner')  # keeps COMMON genes
del adata_diseased_list

# Restore var info for common genes
adata.var = var_info.loc[adata.var_names]

print(adata)
display(adata.obs.head(3))
display(adata.var.head(3))
assert adata.obs_names.is_unique, "ERROR: adata.obs_names are not unique!"



# Case I: healthy Vs Disease - All cells (GEX) - Desseq2 - ct X zone
- Select specified (ct x zone) for each sample
- Merge all selected cells in a single adata
- Pseudobulk
    - Sample level (ignore multiple lib → just check visually)
    - Sum raw counts
    - 1 pseudo_cell per sample
    - Min 10 cells to be a pseudocell
    - Min 3 pseudocell per group
- Run test with formula/cov
    - design = ~ disease_state + log1p_n_genes_by_counts + pct_mito
